In [3]:
from dotenv import load_dotenv
import os
import pandas as pd
import cv2
from easyocr import Reader

In [2]:
# Initialisiere EasyOCR mit der gewünschten Sprache (in dem Fall Englisch)
reader = Reader(['en'], gpu=True)

# Create output list for individual adverts

In [3]:
# Definiere den Pfad zu deinem Zielordner
target_folder = "/home/giulia/Commercial-Brand-Differentiating-Message-Analysis/01_input_frames_all/2016/ADs_IG_2016/AD0417"


### Methode 2: Wasserzeichen abdecken mit schwarzen Box

In [4]:

# Funktion: Wasserzeichen-Bereich abdecken
def cover_watermark(frame, x, y, w, h, method="black"):
    """
    Überdeckt den Wasserzeichen-Bereich in einem Bild.
    Args:
    - frame: Das Bild/Frame (numpy-Array)
    - x, y: Obere linke Ecke des Wasserzeichen-Bereichs
    - w, h: Breite und Höhe des Wasserzeichen-Bereichs
    - method: Methode zum Abdecken ("black" oder "blur")
    """
    roi = frame[y:y+h, x:x+w]
    if method == "black":
        # Füllt den Bereich mit Schwarz
        frame[y:y+h, x:x+w] = (0, 0, 0)
    elif method == "blur":
        # Wendet einen Weichzeichner auf den Bereich an
        frame[y:y+h, x:x+w] = cv2.GaussianBlur(roi, (15, 15), 0)
    return frame


neue procee

In [5]:
def process_frames_with_watermark_removal(frames, x, y, w, h, method="black"):
    """
    Entfernt Wasserzeichen und führt OCR auf allen Frames durch, gruppiert die Ergebnisse nach Frames.
    Args:
        frames (List[Dict]): Eine Liste von Dictionaries mit Frame-Pfad und Werbungsnamen.
        x, y (int): Obere linke Ecke des Wasserzeichen-Bereichs.
        w, h (int): Breite und Höhe des Wasserzeichen-Bereichs.
        method (str): Methode zum Abdecken des Wasserzeichens ("black" oder "blur").

    Returns:
        pd.DataFrame: Ein DataFrame mit den gruppierten OCR-Ergebnissen (Spalten: Frame, Recognized_Text).
    """
    results_list = []

    # Iteriere durch alle Frames
    for frame_info in frames:
        ad_name = frame_info["ad"]
        frame_path = frame_info["frame_path"]

        # Lade das Bild
        image = cv2.imread(frame_path)

        # Abdeckung des Wasserzeichens
        image = cover_watermark(image, x, y, w, h, method)

        # Perform text recognition with EasyOCR (nur auf das abgedeckte Bild)
        ocr_results = reader.readtext(image)

        # Ergebnisse speichern
        for (bbox, text, prob) in ocr_results:
            # Überprüfe, ob der Text gültig ist (nicht leer oder nur Sonderzeichen)
            if text and text.strip() and len(text.strip()) > 1:  # Sicherstellen, dass der Text sinnvoll ist
                results_list.append({
                    "Frame": os.path.basename(frame_path),  # Nur der Dateiname
                    "Recognized_Text": text.strip(),       # Direkter Text ohne zusätzliche Filterung
                })

    # Ergebnisse in DataFrame umwandeln
    results_df = pd.DataFrame(results_list)

    # Gruppiere die Ergebnisse nach Frames
    merged_results = (
        results_df.groupby("Frame")
        .agg({
            "Recognized_Text": lambda x: ", ".join(x),  # Kombiniere alle erkannten Texte pro Frame
        })
        .reset_index()
    )

    return merged_results


### Duplikate entfernen

In [6]:
from collections import Counter
from rapidfuzz import fuzz

# Funktion: Gruppiere Texte nach Ähnlichkeit und wähle den häufigsten
def remove_similar_texts(texts, threshold=80):
    text_groups = []
    for text in texts:
        group_found = False
        for group in text_groups:
            if fuzz.ratio(text, group[0]) >= threshold:  # Überprüfe Ähnlichkeit
                group.append(text)
                group_found = True
                break
        if not group_found:
            text_groups.append([text])
    # Wähle pro Gruppe den häufigsten Text
    return [Counter(group).most_common(1)[0][0] for group in text_groups]


# Funktion: Erstelle die bereinigte Tabelle
def create_cleaned_dataframe(merged_results, ad_column="Frame", text_column="Recognized_Text", threshold=60):
    # Extrahiere den "Ad"-Namen aus den Frame-Namen
    merged_results["ad"] = merged_results[ad_column].apply(lambda x: x.split("_")[0])  
    # Gruppiere nach "ad" und bereinige Texte
    cleaned_data = []
    for ad, group in merged_results.groupby("ad"):
        unique_texts = remove_similar_texts(group[text_column].tolist(), threshold=threshold)
        unique_texts = [text for text in unique_texts if text.strip()]
        combined_text = "; ".join(unique_texts)  # Kombiniere die bereinigten Texte
        cleaned_data.append({"ad": ad, "recognized_text": combined_text})
    
    # Erstelle eine neue DataFrame
    cleaned_df = pd.DataFrame(cleaned_data)
    return cleaned_df


In [7]:
#cleaned_df.to_csv("cleaned_df_box.csv", index=False)

# Main Script

In [8]:
def load_all_frames(input_folder):
    """
    Listet alle Frames aus allen Unterordnern auf.
    Args:
        input_folder (str): Der Pfad zum Hauptordner mit den Frames.
    Returns:
        List[Dict]: Eine Liste von Dictionaries mit Frame-Pfad und zugehörigem Werbungsnamen.
    """
    frames = []
    for root, dirs, files in os.walk(input_folder):
        for file in files:
            if file.endswith((".png", ".jpg")):  # Akzeptiere nur Bilddateien
                ad_name = os.path.basename(root)  # Name des Werbungs-Ordners
                frame_path = os.path.join(root, file)
                frames.append({"ad": ad_name, "frame_path": frame_path})
    return frames

In [9]:
def process_all_ads(input_folder, x, y, w, h, method="black", threshold=70):
    """
    Hauptpipeline, die alle Schritte für die Verarbeitung aller Werbungen durchführt.
    Args:
        input_folder (str): Der Pfad zum Hauptordner mit den Frames.
    Returns:
        pd.DataFrame: Ein bereinigtes DataFrame mit allen Werbungen und kombinierten Texten.
    """
    # Lade alle Frames
    frames = load_all_frames(input_folder)
    
    # Führe OCR auf allen Frames durch
    ocr_results_df = process_frames_with_watermark_removal(frames, x, y, w, h, method)
    
    # Bereinige und kombiniere die Texte
    cleaned_df = create_cleaned_dataframe(ocr_results_df, threshold=threshold)
    
    return cleaned_df

In [10]:
input_folder = os.getenv("INPUT_FRAMES_ALL")  # Oder setze den Pfad direkt: "path/to/frames"

In [11]:
# Hauptskript
x, y, w, h = 62, 545, 235, 60  # Anpassen, falls nötig
method = "black"
threshold = 60

# 1. Lade alle Frames
frames = load_all_frames(input_folder)

print(f"Anzahl der geladenen Frames: {len(frames)}")
print(f"Anzahl der unterschiedlichen Werbungen: {len(set([frame['ad'] for frame in frames]))}")

# 2. Entferne Wasserzeichen und führe OCR aus
ocr_results_df = process_frames_with_watermark_removal(frames, x, y, w, h, method)


Anzahl der geladenen Frames: 64981
Anzahl der unterschiedlichen Werbungen: 537


In [15]:
ocr_results_df

,Frame,Recognized_Text,ad
0,AD0252_Frame_1610.png,Bravery:,AD0252
1,AD0252_Frame_1620.png,Bravery:,AD0252
2,AD0252_Frame_1630.png,Bravery:,AD0252
3,AD0252_Frame_1640.png,Bravery: It'5 what defines us.,AD0252
4,AD0252_Frame_1650.png,Bravery: It's what defines us.,AD0252
...,...,...,...
23657,AD0801_Frame_610.png,FN,AD0801
23658,AD0801_Frame_810.png,"Wea, ;om",AD0801
23659,AD0801_Frame_820.png,WeatherTeche_com,AD0801
23660,AD0801_Frame_830.png,WeatherTeche_com,AD0801


In [ ]:

# 3. Bereinige die Ergebnisse und kombiniere Texte
final_results_df = create_cleaned_dataframe(ocr_results_df, text_column="Recognized_Text", threshold=60)

# 4. Zeige die Ergebnisse
from IPython.display import display
display(final_results_df)


,ad,recognized_text
0,AD0252,Bravery:; Bravery: It'5 what defines us.; Audi...
1,AD0253,"AFF, MEEP; Ficlionalizalion; Acllowallzallou; ..."
2,AD0254,"soluor CNRAL, Sprint, #Mobile:, atet; MCINTRAL..."
3,AD0255,Screen images suulal; 112z8; Fuzt;; 1e04213; s...
4,AD0256,"BECKS, SAPPHII; bEck:, ALC /Vol, BECKS, SAPPHI..."
...,...,...
530,AD0797,"Uber, Eats; Foil, Aluminum; Prop food Do not e..."
531,AD0798,"verizon, 12tia007; Signupat verizon com/SG, 5G..."
532,AD0799,"3i, Go, Le ta, (STop, talae, Wae, Weun; (STop,..."
533,AD0800,"20,2012. Seth T. was struck by lightning:, May..."


Ergebnisse wurden erfolgreich in 'final_ocr_results.csv' gespeichert.


#### Wörter löschen, die nicht im Wörterbuch stehen

In [1]:
import re
import enchant

# Wörterbuch für Englisch
english_dict = enchant.Dict("en_US")

def clean_recognized_text_with_enchant(text):
    """
    Bereinigt erkannte Texte, indem bedeutungslose Symbole, Zeichen und zufällige Wörter entfernt werden.
    Prüft zusätzlich, ob Wörter im Wörterbuch vorhanden sind.
    
    Args:
        text (str): Der erkannte Text.
    Returns:
        str: Der bereinigte Text.
    """
    # Whitelist von häufigen kurzen Wörtern
    whitelist = {"in", "on", "by", "at", "to", "is", "as", "it", "of", "be", "or", "we", "an", "up", "no", "do", "am", "go", "my", "me", "us", "so"}

    # Entferne nicht-alphanumerische Zeichen, außer Leerzeichen und Kommas
    text = re.sub(r"[^\w\s]", "", text)
    print("without_sonderzeichen:", text)

    # Teile den Text in Wörter
    words_in_text = text.split()
    print("split:",words_in_text)
    # Filtere Wörter, die entweder:
    # 1. Im Wörterbuch vorkommen oder
    # 2. In der Whitelist enthalten sind
    meaningful_words = [
        word for word in words_in_text
        if english_dict.check(word) or word.lower() in whitelist
    ]

    # Kombiniere die bereinigten Wörter zurück in einen Text
    cleaned_text = " ".join(meaningful_words)

    return cleaned_text


In [7]:
ocr_to_merge = pd.read_csv("final_ocr_results.csv")


In [8]:
ocr_to_merge["cleaned_text"] = ocr_to_merge["recognized_text"].apply(clean_recognized_text_with_enchant)

#Speichere die Ergebnisse
ocr_to_merge.to_csv("ocr_to_merge.csv", index=False)
print("Ergebnisse wurden erfolgreich in 'ocr_to_merge.csv' gespeichert.")

without_sonderzeichen: Bravery Bravery It5 what defines us Audi CW Truth in Engineering BraveryWins SCORING SUMMARY BALTIMORE PLAYS 6 Art YARDS 51 RAVENS TIME 229 Jin 13YARD TD RECEPTION Away Fiel 22 SCORING SUMMARY BALTIMORE ANQUAN BOLDIN AtH REC TD THIS POSTSEASON MFD Art 4 REC TD IN REG SEASON RAVENS 903 Closed Course obey speed ard traffic laws Aluys PRINCIPAL ONLY PRINCE PathctpaE BHrSoN ISQHIOOL Hu bxdron ISOHOOL Hul
split: ['Bravery', 'Bravery', 'It5', 'what', 'defines', 'us', 'Audi', 'CW', 'Truth', 'in', 'Engineering', 'BraveryWins', 'SCORING', 'SUMMARY', 'BALTIMORE', 'PLAYS', '6', 'Art', 'YARDS', '51', 'RAVENS', 'TIME', '229', 'Jin', '13YARD', 'TD', 'RECEPTION', 'Away', 'Fiel', '22', 'SCORING', 'SUMMARY', 'BALTIMORE', 'ANQUAN', 'BOLDIN', 'AtH', 'REC', 'TD', 'THIS', 'POSTSEASON', 'MFD', 'Art', '4', 'REC', 'TD', 'IN', 'REG', 'SEASON', 'RAVENS', '903', 'Closed', 'Course', 'obey', 'speed', 'ard', 'traffic', 'laws', 'Aluys', 'PRINCIPAL', 'ONLY', 'PRINCE', 'PathctpaE', 'BHrSoN', 'IS